# Residual Network in tensorflow

Import tensorflow module, mnist dataset and matplotlib library for ploting learning history

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pyplot as plt

Load MNIST data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Define methods previously defined for last excercise

In [3]:
def variable_summaries(var, name):
    with tf.name_scope('summaries_{}'.format(name)):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram(name, var)

In [4]:
def convolutional_layer(layer_input, layer_size, activation="linear", stride=1, stddev=0.1, padding='same', 
                        name="convolutional_layer"):
    # Input layer shape
    input_shape = layer_input.get_shape().as_list()[1:]
    input_shape_filters = input_shape[-1]
    
    with tf.name_scope(name):
        weights = tf.Variable(tf.truncated_normal([layer_size[0], layer_size[1], input_shape_filters, 
                                                   layer_size[2]], stddev=stddev))
        bias = tf.Variable(tf.ones([layer_size[2]]) / 10)
        not_activated = tf.nn.conv2d(layer_input, weights, strides=[1, stride, stride, 1], padding=padding.upper())
        not_activated = tf.nn.bias_add(not_activated, bias)

        if activation != 'linear':
            activated_output = getattr(tf.nn, activation)(not_activated)
        else:
            activated_output = not_activated

        #  Get histograms
        variable_summaries(weights, "weights")
        variable_summaries(bias, "biases")
        tf.summary.histogram("activations", activated_output)
    return activated_output
    

In [5]:
def faltten(layer_input, name="flatten"):
    with tf.name_scope(name):
        input_shape = layer_input.get_shape().as_list()[1:]
    return tf.reshape(Y3, shape=[-1, np.prod(input_shape)])

In [6]:
def fully_connected_layer(layer_input, output_size, activation="linear", name="fully_connected"):
    input_shape = layer_input.get_shape().as_list()[1:]
    with tf.name_scope(name):
        weights = tf.Variable(tf.truncated_normal([int(np.prod(input_shape)), output_size], stddev=0.1))
        bias = tf.Variable(tf.ones([output_size]) / 10)
        
        not_activated = tf.matmul(layer_input, weights)
        not_activated = tf.nn.bias_add(not_activated, bias)

        if activation != 'linear':
            activated_output = getattr(tf.nn, activation)(not_activated)
        else:
            activated_output = not_activated
        variable_summaries(weights, "weights")
        variable_summaries(bias, "biases")
        tf.summary.histogram("activations", activated_output)
    return activated_output

## Residual Networks

Residual Network is type of network released by Microsoft. Residual Networks was used as a flag AI product for Imagenet-1k competition. Team won this competition in classification domain and also in other domains like detection and semantic segmentation. From this moment Residual Networks are widely used in computer vision domain and in other fields connected with machine learning (e.g Speech Recognition, Natural Language Processing).

![title](imgs/scores.png)

## Degradation Problem

With network depth increasing, accuracy gets saturated (which might be unsurprising) and then degrades rapidly. Unexpectedly, such **degradation** is not caused by **overfitting**, and adding more layers to a suitably deep model leads to higher training error.

**Residual Networks** use very simple trick to prevent such situation, they introduce special **identity connction**. If model claims that block of layer is unnecessary it will use **identity mapping** from input to output.
Otherwise learned "residual" will be added to input (Instead calculating whole output).

![title](imgs/residual.png)

In [7]:
def residual_block(layer_input, out_channel, stride, is_train, name="residual_unit"):
    with tf.name_scope(name):
        if stride != 1:
            shortcut = convolutional_layer(layer_input, [1, 1, out_channel], activation="relu", 
                                           stride=stride, padding="same", stddev=0.1, name="convo_bottle_{}".format(name))
        else:
            shortcut = layer_input
        x_1 = convolutional_layer(layer_input, [3, 3, out_channel], activation="linear", 
                                  stride=stride, padding="same", stddev=0.1, name="convo_1_{}".format(name))
        x_2 = tf.contrib.layers.batch_norm(x_1, decay=0.99, updates_collections=None, 
                                           epsilon=0.001,
                                           scale=True,
                                           center=True,
                                           is_training=is_train,
                                           scope="bn_1_{}".format(name))
        x_3 = tf.nn.relu(x_2)
        x_4 = convolutional_layer(x_3, [3, 3, out_channel], activation="linear", 
                                  stride=1, padding="same", stddev=0.1, name="convo_2_{}".format(name))
        x_5 = tf.contrib.layers.batch_norm(x_4, decay=0.99, updates_collections=None, 
                                           epsilon=0.001,
                                           scale=True,
                                           center=True,
                                           is_training=is_train,
                                           scope="bn_2_{}".format(name))
        x_6 = x_5 + shortcut
        x_7 = tf.nn.relu(x_6)
    return x_7

Define Input placeholder and is_train placeholder

In [8]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
is_train = tf.placeholder(tf.bool)

Define network body

In [9]:
Y1 = residual_block(X, out_channel=64, stride=2, is_train=is_train, name="residual_unit_1")
Y2 = residual_block(Y1, out_channel=64, stride=1, is_train=is_train, name="residual_unit_2")

Y3 = residual_block(Y2, out_channel=128, stride=2, is_train=is_train, name="residual_unit_3")
Y4 = residual_block(Y3, out_channel=128, stride=1, is_train=is_train, name="residual_unit_4")

Y5 = residual_block(Y4, out_channel=256, stride=2, is_train=is_train, name="residual_unit_5")
Y6 = residual_block(Y5, out_channel=256, stride=1, is_train=is_train, name="residual_unit_6")

Y7 = residual_block(Y6, out_channel=512, stride=2, is_train=is_train, name="residual_unit_7")
Y8 = residual_block(Y7, out_channel=512, stride=1, is_train=is_train, name="residual_unit_8")

YY = faltten(Y3, name="flatten")
Y4 = fully_connected_layer(YY, output_size=200, activation="relu", name="fc_1")
Y4_d = tf.nn.dropout(Y4, 0.75)
Ylogits = fully_connected_layer(Y4_d, 10, activation="linear", name="fc_1")
Y = tf.nn.softmax(Ylogits)

Placeholder for Y labels

In [10]:
Y_pred = tf.placeholder(tf.float32, [None, 10])

Define cross entropy [*loss function*]

In [11]:
with tf.name_scope("Cross_entropy"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_pred)
    cross_entropy = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("cross_entropy", cross_entropy)

In [12]:
with tf.name_scope("Accuracy"):
    is_correct = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_pred, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

Define Gradient Descent Optimizer

In [13]:
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

In [14]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [15]:
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./logs")
writer.add_graph(sess.graph)

In [ ]:
test_accuracy = []
test_cross = []
for i in range(2000):
    # Load batch
    batch_X, batch_Y = mnist.train.next_batch(100)
    batch_X = batch_X.reshape((100, 28, 28, 1))
    train_data = {X: batch_X, Y_pred: batch_Y, is_train: True}

    # Train
    sess.run(train_step, feed_dict=train_data)
    a_train, c_train = sess.run([accuracy, cross_entropy], feed_dict=train_data)

    test_batch_x = mnist.test.images
    test_batch_x = test_batch_x.reshape((10000, 28, 28, 1))
    test_data = {X: test_batch_x, Y_pred: mnist.test.labels, is_train: False}
    a_test, c_test = sess.run([accuracy, cross_entropy], feed_dict=test_data)
    print("Train data Accuracy: {}, Train data Cross Entropy: {}".format(a_train, c_train))
    print("Test data Accuracy: {}, Test data Cross Entropy: {}".format(a_test, c_test))

    # Save summary
    if i % 5 == 0:
        sum_res = sess.run(merged_summary, train_data)
        writer.add_summary(sum_res, i)
    
    # Append to list
    test_accuracy.append(a_test)
    test_cross.append(c_test)

Train data Accuracy: 0.09000000357627869, Train data Cross Entropy: 9.026470184326172
Test data Accuracy: 0.09989999979734421, Test data Cross Entropy: 35.00803756713867
Train data Accuracy: 0.11999999731779099, Train data Cross Entropy: 7.956079006195068
Test data Accuracy: 0.09889999777078629, Test data Cross Entropy: 27.911331176757812
Train data Accuracy: 0.11999999731779099, Train data Cross Entropy: 7.465938568115234
Test data Accuracy: 0.10329999774694443, Test data Cross Entropy: 24.367347717285156
Train data Accuracy: 0.1599999964237213, Train data Cross Entropy: 7.07771110534668
Test data Accuracy: 0.10320000350475311, Test data Cross Entropy: 21.789566040039062
Train data Accuracy: 0.10999999940395355, Train data Cross Entropy: 6.85464334487915
Test data Accuracy: 0.10670000314712524, Test data Cross Entropy: 19.6999454498291
Train data Accuracy: 0.20000000298023224, Train data Cross Entropy: 6.2755255699157715
Test data Accuracy: 0.1088000014424324, Test data Cross Entropy:

Train data Accuracy: 0.6600000262260437, Train data Cross Entropy: 1.3928749561309814
Test data Accuracy: 0.2903999984264374, Test data Cross Entropy: 2.365525722503662
Train data Accuracy: 0.5899999737739563, Train data Cross Entropy: 1.8129326105117798
Test data Accuracy: 0.28780001401901245, Test data Cross Entropy: 2.3724465370178223
Train data Accuracy: 0.6399999856948853, Train data Cross Entropy: 1.7202255725860596
Test data Accuracy: 0.27889999747276306, Test data Cross Entropy: 2.385848045349121
Train data Accuracy: 0.6800000071525574, Train data Cross Entropy: 1.3955365419387817
Test data Accuracy: 0.2791999876499176, Test data Cross Entropy: 2.3913440704345703
Train data Accuracy: 0.6800000071525574, Train data Cross Entropy: 1.1240023374557495
Test data Accuracy: 0.2809999883174896, Test data Cross Entropy: 2.4031124114990234
Train data Accuracy: 0.6200000047683716, Train data Cross Entropy: 1.6965229511260986
Test data Accuracy: 0.2856999933719635, Test data Cross Entropy:

Train data Accuracy: 0.699999988079071, Train data Cross Entropy: 0.9774281978607178
Test data Accuracy: 0.30219998955726624, Test data Cross Entropy: 2.288100004196167
Train data Accuracy: 0.75, Train data Cross Entropy: 0.9824953675270081
Test data Accuracy: 0.31439998745918274, Test data Cross Entropy: 2.271831750869751
Train data Accuracy: 0.7200000286102295, Train data Cross Entropy: 0.8100989460945129
Test data Accuracy: 0.30390000343322754, Test data Cross Entropy: 2.286597967147827
Train data Accuracy: 0.6700000166893005, Train data Cross Entropy: 1.2555124759674072
Test data Accuracy: 0.31360000371932983, Test data Cross Entropy: 2.246642589569092
Train data Accuracy: 0.7200000286102295, Train data Cross Entropy: 0.8899381160736084
Test data Accuracy: 0.31949999928474426, Test data Cross Entropy: 2.2435996532440186
Train data Accuracy: 0.7200000286102295, Train data Cross Entropy: 0.9326721429824829
Test data Accuracy: 0.31779998540878296, Test data Cross Entropy: 2.2356920242

Train data Accuracy: 0.7699999809265137, Train data Cross Entropy: 0.8024693131446838
Test data Accuracy: 0.5013999938964844, Test data Cross Entropy: 1.516371488571167
Train data Accuracy: 0.8299999833106995, Train data Cross Entropy: 0.5697171688079834
Test data Accuracy: 0.5101000070571899, Test data Cross Entropy: 1.5014045238494873
Train data Accuracy: 0.7300000190734863, Train data Cross Entropy: 0.9397321343421936
Test data Accuracy: 0.5117999911308289, Test data Cross Entropy: 1.4825865030288696
Train data Accuracy: 0.8299999833106995, Train data Cross Entropy: 0.4751955270767212
Test data Accuracy: 0.519599974155426, Test data Cross Entropy: 1.4550046920776367
Train data Accuracy: 0.8199999928474426, Train data Cross Entropy: 0.6285408139228821
Test data Accuracy: 0.5335000157356262, Test data Cross Entropy: 1.4229373931884766
Train data Accuracy: 0.8600000143051147, Train data Cross Entropy: 0.44542908668518066
Test data Accuracy: 0.5389999747276306, Test data Cross Entropy: 

Train data Accuracy: 0.7799999713897705, Train data Cross Entropy: 0.8551335334777832
Test data Accuracy: 0.7311000227928162, Test data Cross Entropy: 0.8393574357032776
Train data Accuracy: 0.8899999856948853, Train data Cross Entropy: 0.5170193910598755
Test data Accuracy: 0.7347999811172485, Test data Cross Entropy: 0.8185949921607971
Train data Accuracy: 0.7599999904632568, Train data Cross Entropy: 0.9927038550376892
Test data Accuracy: 0.7390000224113464, Test data Cross Entropy: 0.8112250566482544
Train data Accuracy: 0.7799999713897705, Train data Cross Entropy: 0.737053632736206
Test data Accuracy: 0.7329000234603882, Test data Cross Entropy: 0.8265449404716492
Train data Accuracy: 0.8399999737739563, Train data Cross Entropy: 0.5499435663223267
Test data Accuracy: 0.7434999942779541, Test data Cross Entropy: 0.8040904402732849
Train data Accuracy: 0.8600000143051147, Train data Cross Entropy: 0.5152409076690674
Test data Accuracy: 0.7476000189781189, Test data Cross Entropy: 

...

Train data Accuracy: 0.9100000262260437, Train data Cross Entropy: 0.1579645425081253
Test data Accuracy: 0.9452000260353088, Test data Cross Entropy: 0.18485519289970398
Train data Accuracy: 0.9300000071525574, Train data Cross Entropy: 0.16199573874473572
Test data Accuracy: 0.9447000026702881, Test data Cross Entropy: 0.18279491364955902
Train data Accuracy: 0.949999988079071, Train data Cross Entropy: 0.15275470912456512
Test data Accuracy: 0.9422000050544739, Test data Cross Entropy: 0.17909450829029083
Train data Accuracy: 0.949999988079071, Train data Cross Entropy: 0.16848434507846832
Test data Accuracy: 0.9434999823570251, Test data Cross Entropy: 0.18303880095481873
Train data Accuracy: 1.0, Train data Cross Entropy: 0.0421869233250618
Test data Accuracy: 0.9423999786376953, Test data Cross Entropy: 0.18812698125839233
Train data Accuracy: 0.949999988079071, Train data Cross Entropy: 0.20473113656044006
Test data Accuracy: 0.9478999972343445, Test data Cross Entropy: 0.179314

Train data Accuracy: 0.8999999761581421, Train data Cross Entropy: 0.3341670334339142
Test data Accuracy: 0.9470999836921692, Test data Cross Entropy: 0.17130117118358612
Train data Accuracy: 0.9700000286102295, Train data Cross Entropy: 0.14707425236701965
Test data Accuracy: 0.9434999823570251, Test data Cross Entropy: 0.18071676790714264
Train data Accuracy: 0.9300000071525574, Train data Cross Entropy: 0.2284998893737793
Test data Accuracy: 0.9451000094413757, Test data Cross Entropy: 0.17987126111984253
Train data Accuracy: 0.9800000190734863, Train data Cross Entropy: 0.09396590292453766
Test data Accuracy: 0.9444000124931335, Test data Cross Entropy: 0.1862172782421112
Train data Accuracy: 0.9399999976158142, Train data Cross Entropy: 0.25058692693710327
Test data Accuracy: 0.9444000124931335, Test data Cross Entropy: 0.1908036172389984
Train data Accuracy: 0.949999988079071, Train data Cross Entropy: 0.16518110036849976
Test data Accuracy: 0.9419999718666077, Test data Cross En

Train data Accuracy: 0.9599999785423279, Train data Cross Entropy: 0.13410383462905884
Test data Accuracy: 0.9505000114440918, Test data Cross Entropy: 0.16914714872837067
Train data Accuracy: 0.9300000071525574, Train data Cross Entropy: 0.1452367752790451
Test data Accuracy: 0.9496999979019165, Test data Cross Entropy: 0.1645032912492752
Train data Accuracy: 0.9700000286102295, Train data Cross Entropy: 0.06141134351491928
Test data Accuracy: 0.9509000182151794, Test data Cross Entropy: 0.17064282298088074
Train data Accuracy: 0.9399999976158142, Train data Cross Entropy: 0.16338668763637543
Test data Accuracy: 0.9480999708175659, Test data Cross Entropy: 0.1736096441745758
Train data Accuracy: 1.0, Train data Cross Entropy: 0.02677030861377716
Test data Accuracy: 0.9491999745368958, Test data Cross Entropy: 0.17485392093658447
Train data Accuracy: 0.949999988079071, Train data Cross Entropy: 0.12920676171779633
Test data Accuracy: 0.9487000107765198, Test data Cross Entropy: 0.16976

Train data Accuracy: 0.9700000286102295, Train data Cross Entropy: 0.11897184699773788
Test data Accuracy: 0.95169997215271, Test data Cross Entropy: 0.16142907738685608
Train data Accuracy: 0.9800000190734863, Train data Cross Entropy: 0.046183206140995026
Test data Accuracy: 0.9502999782562256, Test data Cross Entropy: 0.16206684708595276
Train data Accuracy: 0.9599999785423279, Train data Cross Entropy: 0.1384706050157547
Test data Accuracy: 0.951200008392334, Test data Cross Entropy: 0.16416800022125244
Train data Accuracy: 0.9599999785423279, Train data Cross Entropy: 0.11072997748851776
Test data Accuracy: 0.9488000273704529, Test data Cross Entropy: 0.17260296642780304
Train data Accuracy: 0.9599999785423279, Train data Cross Entropy: 0.09728212654590607
Test data Accuracy: 0.9495999813079834, Test data Cross Entropy: 0.16910597681999207
Train data Accuracy: 0.9300000071525574, Train data Cross Entropy: 0.22589249908924103
Test data Accuracy: 0.9495000243186951, Test data Cross 

Train data Accuracy: 0.949999988079071, Train data Cross Entropy: 0.16595110297203064
Test data Accuracy: 0.9495999813079834, Test data Cross Entropy: 0.16353914141654968
Train data Accuracy: 0.9300000071525574, Train data Cross Entropy: 0.14926040172576904
Test data Accuracy: 0.9503999948501587, Test data Cross Entropy: 0.16412337124347687
Train data Accuracy: 0.9700000286102295, Train data Cross Entropy: 0.06512830406427383
Test data Accuracy: 0.95169997215271, Test data Cross Entropy: 0.15527111291885376
Train data Accuracy: 0.9800000190734863, Train data Cross Entropy: 0.09692645072937012
Test data Accuracy: 0.947700023651123, Test data Cross Entropy: 0.16922499239444733
Train data Accuracy: 0.9599999785423279, Train data Cross Entropy: 0.13123823702335358
Test data Accuracy: 0.9495000243186951, Test data Cross Entropy: 0.16667082905769348
Train data Accuracy: 0.9200000166893005, Train data Cross Entropy: 0.17130404710769653
Test data Accuracy: 0.9502999782562256, Test data Cross E

In [ ]:
print("Last Accuracy: {}, Maximum Accuracy: {}".format(test_accuracy[-1], np.max(test_accuracy)))